# Recommendation with features

In [ ]:
!git clone https://github.com/massquantity/LibRecommender.git

fatal: destination path 'LibRecommender' already exists and is not an empty directory.


In [ ]:
cd LibRecommender

/content/LibRecommender


In [ ]:
!pip install .

Processing /content/LibRecommender
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for LibRecommender: filename=LibRecommender-1.4.0-cp310-cp310-linux_x86_64.whl size=2095035 sha256=c87e84aceebf15e142c885841408ab222fb8320e4bc1c68f4afea187019f3096
  Stored in directory: /tmp/pip-ephem-wheel-cache-qhvu9p7_/wheels/77/1d/d9/4fb4b7f202e80009c1648a4e31c56ec57f28d01b42649b5883
Successfully built LibRecommender
  Attempting uninstall: LibRecommender
    Found existing installation: LibRecommender 1.4.0
    Uninstalling LibRecommender-1.4.0:
      Successfully uninstalled LibRecommender-1.4.0


In [ ]:
import numpy as np
import pandas as pd
from libreco.data import split_by_ratio_chrono, DatasetFeat
from libreco.algorithms import YouTubeRanking  # feat data, algorithm YouTubeRanking

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
data = pd.read_csv("examples/sample_data/sample_movielens_merged.csv", sep=",", header=0)
data.head()

,user,item,label,time,sex,age,occupation,genre1,genre2,genre3
0,4617,296,2,964138229,F,25,6,crime,drama,missing
1,1298,208,4,974849526,M,35,6,action,adventure,missing
2,4585,1769,4,964322774,M,35,7,action,thriller,missing
3,3706,1136,5,966376465,M,25,12,comedy,missing,missing
4,2137,1215,3,974640099,F,1,10,action,adventure,comedy


In [ ]:
# split into train and test data based on time
train_data, test_data = split_by_ratio_chrono(data, test_size=0.2)

In [ ]:
# specify complete columns information
sparse_col = ["sex", "occupation", "genre1", "genre2", "genre3"]
dense_col = ["age"]
user_col = ["sex", "age", "occupation"]
item_col = ["genre1", "genre2", "genre3"]

In [ ]:
train_data, data_info = DatasetFeat.build_trainset(
    train_data, user_col, item_col, sparse_col, dense_col
)
test_data = DatasetFeat.build_testset(test_data)
print(data_info)  # n_users: 5962, n_items: 3226, data sparsity: 0.4185 %

n_users: 5953, n_items: 3209, data density: 0.4213 %


In [ ]:
ytb_ranking = YouTubeRanking(
    task="ranking",
    data_info=data_info,
    embed_size=16,
    n_epochs=3,
    lr=1e-4,
    batch_size=512,
    use_bn=True,
    hidden_units=(128, 64, 32),
)
ytb_ranking.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    shuffle=True,
    eval_data=test_data,
    metrics=["loss", "roc_auc", "precision", "recall", "map", "ndcg"],
)

Training start time: 2024-02-14 11:32:48


/content/LibRecommender/libreco/layers/dense.py:31: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
Instructions for updating:
Colocations handled automatically by placer.
/content/LibRecommender/libreco/layers/dense.py:39: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)


total params: 177,521 | embedding params: 148,145 | network params: 29,376


train: 100%|██████████| 315/315 [00:06<00:00, 46.78it/s]


Epoch 1 elapsed: 6.745s
	 train_loss: 0.6981


...

	 eval log_loss: 0.6521
	 eval roc_auc: 0.6646
	 eval precision@10: 0.0039
	 eval recall@10: 0.0135
	 eval map@10: 0.0107
	 eval ndcg@10: 0.0171


train: 100%|██████████| 315/315 [00:03<00:00, 81.28it/s]


Epoch 3 elapsed: 3.886s
	 train_loss: 0.5792


eval_listwise: 100%|██████████| 2446/2446 [00:56<00:00, 43.49it/s]


	 eval log_loss: 0.6224
	 eval roc_auc: 0.7208
	 eval precision@10: 0.0050
	 eval recall@10: 0.0193
	 eval map@10: 0.0147
	 eval ndcg@10: 0.0225


In [ ]:
# predict preference of user 2211 to item 110
ytb_ranking.predict(user=2211, item=110)

array([0.8145978], dtype=float32)

## Recommend to user 2211 with diff params

In [ ]:
# recommend 7 items for user 2211
ytb_ranking.recommend_user(user=2211, n_rec=7)

{2211: array([ 527, 1270, 2716, 1294,  541, 2944,  260])}

In [ ]:
ytb_ranking.recommend_user(user=2211, n_rec=7, cold_start="popular")

{2211: array([ 527, 1270, 2716, 1294,  541, 2944,  260])}

In [ ]:
ytb_ranking.recommend_user(user=2211, n_rec=7, cold_start="average")

{2211: array([ 527, 1270, 2716, 1294,  541, 2944,  260])}

In [ ]:
ytb_ranking.recommend_user(user=2211, n_rec=7, cold_start="popular", filter_consumed=True)

{2211: array([ 527, 1270, 2716, 1294,  541, 2944,  260])}

## Recommend to user 1121 with diff params

In [ ]:
ytb_ranking.recommend_user(user=1121, n_rec=7, cold_start="popular")

{1121: array([ 589,  527, 1240, 2571, 1270, 1196, 1193])}

In [ ]:
ytb_ranking.recommend_user(user=1121, n_rec=7, cold_start="average")

{1121: array([ 589,  527, 1240, 2571, 1270, 1196, 1193])}

In [ ]:
# cold-start prediction
ytb_ranking.predict(user="ccc", item="not item", cold_start="average")

Detect 1 unknown interaction(s), position: [0]


array([0.76414376], dtype=float32)

In [ ]:
# cold-start recommendation
ytb_ranking.recommend_user(user="are we good?", n_rec=7, cold_start="popular")

Detect unknown user: are we good?


{'are we good?': array([ 480, 1573,   50, 1193, 1193, 1393,  480])}